#### This assignment may be worked individually or in pairs. Enter your name/s here:
    

In [1]:
#Kayla Han

# Assignment 2: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1150` records and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

0) The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.

1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

8-15) contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the 
number of lesions with the diameter of the ROI to compensate different image sizes. 

16) The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patient's condition. This feature is also normalized with the diameter of the ROI.

17) The diameter of the optic disc. 

18) The binary result of the AM/FM-based classification.

19) Class label. 1 = contains signs of Diabetic Retinopathy, 0 = no signs of Diabetic Retinopathy.

#### Implementation: 
The function prototypes are given to you, please don't change those. You can add additional helper functions if needed. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 values. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5. For binary features [0/1] the threshold value will be 0.5.
- All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [1]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
from math import log2
from collections import Counter

In [2]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    def __init__(self, label, features):
        self.label = label # the classification label of this data point
        self.features = features

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [3]:
def get_data(filename):
    data = []
#     your code goes here
    file = open(filename, 'r')
    lines = file.readlines()
    count = 0
    for line in lines:
        line = [float(val) for val in line.strip().split(",")]
        label = line[19]
        del line[19]
        data.append(DataPoint(label, line))
    return data

In [31]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self, level=0):    # for debugging purposes
        print(self)
        if self.is_leaf:
            print ('-'*level + 'Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('-'*level + 'Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree(level+1)
            self.right_child.printTree(level+1)

Q2. Implement the function `make_prediction` that takes the decision tree root and a `DataPoint` instance and returns the prediction label.

In [5]:
def make_prediction(tree_root, data_point):
#     your code goes here
    cur_node = tree_root
    while not cur_node.is_leaf:
        if data_point.features[cur_node.feature_idx] < cur_node.thresh_val:
            cur_node = cur_node.left_child
        else:
            cur_node = cur_node.right_child
    return cur_node.prediction

Q3. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [6]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []
#     your code goes here
    for line in data:
        if line.features[feature_idx] < threshold:
            left_split.append(line)
        else:
            right_split.append(line)
    return (left_split, right_split)

Q4. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [7]:
def calc_entropy(data):
    entropy = 0.0
#     your code goes here
    total_points = len(data)
    yes = 0
    no = 0
    for line in data:
        #print("A: ", line.label)
        if line.label > 0.0:
            yes += 1
        else:
            no += 1
    yes_entropy = 0.0
    no_entropy = 0.0
    yes_frac = yes/total_points
    no_frac = no/total_points
    if yes_frac != 0.0 and no_frac != 0.0:
        entropy = -(yes_frac)*log2(yes_frac) - (no_frac)*log2(no_frac)
    return entropy

Q5. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [8]:
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = None
#     your code goes here
    parent_entropy = calc_entropy(data)

    cur_thresh = 0
    data.sort(key=lambda point: point.features[feature_idx])
    pos = 1
    cur_line = data[0]
    while pos < len(data):
        #print(data[pos].features[feature_idx])
        cur_thresh = (data[pos].features[feature_idx] + cur_line.features[feature_idx])/2
       # print(cur_thresh)
        if (cur_thresh != data[pos].features[feature_idx]):
            split_data_left, split_data_right = split_dataset(data, feature_idx, cur_thresh)
            left_frac = len(split_data_left)/len(data)
            right_frac = len(split_data_right)/len(data)
            gain = parent_entropy - ((left_frac * calc_entropy(split_data_left)) + (right_frac * calc_entropy(split_data_right)))
          #  print("meow: ", gain)
            if (gain > best_info_gain):
                best_thresh = cur_thresh
                best_info_gain = gain
        cur_line = data[pos]
        pos += 1
        
    if best_thresh == None:
        return (None, None)
    
    else:
        return (best_info_gain, best_thresh)

Q6. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [9]:
def identify_best_split(data):
    if len(data) < 2:
        return (None, None)
    best_feature = None
    best_thresh = None
#     your code goes here
    best_info_gain = None
    cur_info_gain = None
    cur_thresh = None
        
    feature_num = len(data[0].features)
    for feature in range(feature_num):
        cur_info_gain, cur_thresh = calc_best_threshold(data, feature)
        if cur_info_gain == None and cur_thresh == None:
            return (None, None)
        if best_info_gain == None or cur_info_gain > best_info_gain:
            best_thresh = cur_thresh
            best_info_gain = cur_info_gain
            best_feature = feature
    return (best_feature, best_thresh)

Q7. Implement the function `create_leaf_node` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node.

In [33]:
def create_leaf_node(data):
#     your code goes here
    new_node = TreeNode()
    new_node.is_leaf = True
    labels = [d.label for d in data]
    counts = Counter(labels)
    new_node.prediction = max(counts)
    return new_node

Q8. Implement the `create_decision_tree` function. `max_levels` denotes the maximum height of the tree (for example if `max_levels = 1` then the decision tree will only contain the leaf node at the root. [Hint: this is where the recursion happens.]

In [34]:
def create_decision_tree(data, max_levels):
#     your code goes here
    root_node = addHelper(data, max_levels)
    return root_node
        
def addHelper(data, max_levels):
   # print("best split: ", identify_best_split(data))
    print("Before if")
    if (max_levels == 1 or identify_best_split(data) == (None, None)):
        new_node = create_leaf_node(data)
        return new_node
    else:
        print("after if")
        # find best split
        # partition records into smaller subsets
        part_feature, part_thresh = identify_best_split(data)
       # print("Splitting on feature: " , part_feature)
       # print("Splitting with threshold: ", part_thresh)
        #cur_node.feature_idx = part_feature
        new_node = TreeNode()
        new_node.is_leaf = False
        new_node.feature_idx = part_feature
        new_node.thresh_val = part_thresh
       # cur_node.thresh_val = part_thresh
        part_data_left, part_data_right = split_dataset(data, part_feature, part_thresh)
        # child node created for each partition of the data, apply algo to each child
        print("before recursion")
        new_node.left_child = addHelper(part_data_left, max_levels - 1)
        new_node.right_child = addHelper(part_data_right, max_levels - 1)
        print("after recursion")
        print(new_node)
        return new_node

Q9. Given a test set, the function `calc_accuracy` returns the accuracy of the classifier. You'll use the `make_prediction` function for this.

In [12]:
def calc_accuracy(tree_root, data):
#     your code goes here
    accuracy = 0.0
    correct = 0
    total = len(data)
    for line in data:
        pred = make_prediction(tree_root, line)
        if pred == line.label:
            correct += 1
    return correct / total

Q10. Keeping the `max_levels` parameter as 10, use 5-fold cross validation to measure the accuracy of the model. Print the accuracy of the model.

In [35]:
# edit the code here - this is just a sample to get you started
import time

d = get_data("messidor_features.txt")

# partition data into train_set and test_set
num_per_partition = len(d) / 5

train_set = d[0:1150]
test_set = d[920:1150]

print ('Training set size:', len(train_set))
print ('Test set size    :', len(test_set))

# the timer is just for fun! you will NOT be graded on runtime
start = time.time()

# create the decision tree
tree = create_decision_tree(train_set, 10)

end = time.time()
print ('Time taken:', end - start)

# calculate the accuracy of the tree
accuracy = calc_accuracy(tree, test_set)
print ('The accuracy on the test set is ', str(accuracy * 100.0))
tree.printTree()

Training set size: 1150
Test set size    : 230
Before if
after if
before recursion
Before if
after if
before recursion
Before if
after if
before recursion
Before if
after if
before recursion
Before if
after if
before recursion
Before if
Before if
after if
before recursion
Before if
after if
before recursion
Before if
after if
before recursion
Before if
after if
before recursion
Before if
Before if
after recursion
Before if
after recursion
Before if
after recursion
Before if
after recursion
after recursion
Before if
after recursion
Before if
after recursion
Before if
after recursion
Before if
after recursion
Time taken: 9.618091106414795
The accuracy on the test set is  50.43478260869565
Internal Node:  splits on feature 14 with threshold 0.0203705
-Internal Node:  splits on feature 2 with threshold 55.5
--Internal Node:  splits on feature 8 with threshold 127.805703
---Internal Node:  splits on feature 11 with threshold 0.040542499999999995
----Internal Node:  splits on feature 17 with

SyntaxError: invalid syntax (2127024793.py, line 30)

In [36]:
#TEST CODE

## SMALL DATASET
print('\nSMALL DATASET')
temp_data = get_data("sample_train.txt")
print(f'Length of dataset: {len(temp_data)}')

print(f'Entropy of dataset: {calc_entropy(temp_data)}')

best_gain, best_thresh = calc_best_threshold(temp_data, 3)
print("Best thresh:", best_thresh)
print("Best gain:", best_gain)

feat, thresh = identify_best_split(temp_data)
print(f"Best split: feature_index={feat}, thresh={thresh}")

## FULL DATASET
print('\nFULL DATASET')
full_data = get_data("messidor_features.txt")
print(f'Length of dataset: {len(full_data)}')

print(f'Entropy of dataset: {calc_entropy(full_data)}')

best_gain, best_thresh = calc_best_threshold(full_data, 3)
print("Best thresh:", best_thresh)
print("Best gain:", best_gain)

feat, thresh = identify_best_split(full_data)
print(f"Best split: feature_index={feat}, thresh={thresh}")

print("Create Tree:")
temp_tree = create_decision_tree(temp_data, 10)
temp_tree.printTree()


SMALL DATASET


FileNotFoundError: [Errno 2] No such file or directory: 'sample_train.txt'

In [ ]:
run code snippetVisit Manage Class to disable runnable code snippets×
These are the answers you should get:

SMALL DATASET
Length of dataset: 15
Entropy of dataset: 0.9709505944546686
Best thresh: 35.0
Best gain: 0.18580516288960103
Best split: feature_index=6, thresh=25.5

FULL DATASET
Length of dataset: 1150
Entropy of dataset: 0.9971705766292643
Best thresh: 62.5
Best gain: 0.04595772209113502
Best split: feature_index=14, thresh=0.0203705